In [1]:
from torch_geometric.datasets import PPI
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from torch_geometric.loader import DataLoader

train_dataset = PPI(root='/tmp/PPI', split="train")
val_dataset = PPI(root='/tmp/PPI', split="val")
test_dataset = PPI(root='/tmp/PPI', split="test")
train_dataset.transform = T.NormalizeFeatures()
val_dataset.transform = T.NormalizeFeatures()
test_dataset.transform = T.NormalizeFeatures()

In [2]:
data = []
data.extend(train_dataset)
data.extend(val_dataset)

In [3]:
from AutoTune2 import user_function
from xgboost import XGBClassifier
import time
from sklearn.metrics import f1_score	
from EnsembleFramework import Framework

def train_and_evaluate(train_set, test_set):
    hops =  [3]
    framework = Framework(hops_list= hops,
                          clfs=[XGBClassifier(**{'objective': 'binary:logistic',
 'base_score': None,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': 'cuda:2',
 'enable_categorical': False,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': 2,
 'max_depth': 5,
 'max_leaves': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 1200,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': 0.539620964294334,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9375310941231909,
 'tree_method': 'hist',
 'validate_parameters': None,
 'verbosity': None,
 'alpha': 0.24419435164444453,
 'eta': 0.2431766543218163,
                            #'early_stopping_rounds': 10,
                            #"eval_metric":"error"
                            })],
                          attention_configs=[ {'inter_layer_normalize': False,
    'use_pseudo_attention': True,
    'cosine_eps': 0.01,
    'dropout_attn': None} for i in hops],
                          handle_nan=0.0,
                          gpu_idx=0,
                          user_functions=[user_function for i in hops],
                          multi_target_class=True
                          )
    start = time.time()
    framework.fit(X_train=train_set.x, edge_index=train_set.edge_index,y_train=train_set.y, train_mask=torch.ones(train_set.x.shape[0]).type(torch.bool),
                     kwargs_multi_clf_list=[{"n_jobs":-1}], transform_kwargs_fit = None)#transform_kwargs_fit
    elapsed_time = time.time() - start
    pred = framework.predict(test_set.x, test_set.edge_index, torch.ones(test_set.x.shape[0]).type(torch.bool))
    score = f1_score(test_set.y, pred, average ="micro")
    print(f"F1-score: {score}; Elapsed time: {elapsed_time} s.")
    return elapsed_time, score

In [4]:
from sklearn.model_selection import KFold
import torch 

kf = KFold(n_splits=5)
scores = []
times = []
for i, (train_index, test_index) in enumerate(kf.split(data)):
    train_data = [data[i] for i in train_index]
    test_data = [data[i] for i in test_index]
    train_loader = iter(DataLoader(train_data, batch_size=len(test_index)))
    test_loader = iter(DataLoader(test_data, batch_size=len(test_index)))

    train_set = next(train_loader)
    test_set = next(test_loader)

    elapsed_time, score = train_and_evaluate(train_set, test_set)
    scores.append(score)
    times.append(elapsed_time)

/home/dwalke/git/graph_aware_ml/EnsembleFramework.py:159: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:230.)
  summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")
/home/dwalke/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:07:50] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:2, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


F1-score: 0.9138002961361286; Elapsed time: 503.11538553237915 s.
F1-score: 0.863897089771427; Elapsed time: 488.6962630748749 s.
F1-score: 0.8472356529732697; Elapsed time: 487.51142978668213 s.
F1-score: 0.8835232916925148; Elapsed time: 490.66277837753296 s.
F1-score: 0.8049621590746824; Elapsed time: 486.8423104286194 s.


In [ ]:
import torch
from torch import nn
from torch_geometric.nn import GATConv
from torch.nn import Linear
import torch.nn.functional as F
class GNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, heads, dropout = .2):
        super(GNN, self).__init__()
        self.dropout = dropout
        self.conv1 = GATConv(in_dim, hidden_dim, add_self_loops=True, heads = heads[0], dropout = dropout, concat=True)
        self.lin1 = Linear(in_dim, hidden_dim*heads[0])
        self.conv2 = GATConv(hidden_dim*heads[0], hidden_dim, add_self_loops=True,heads = heads[1], dropout = dropout, concat=True)
        self.lin2 = Linear(hidden_dim*heads[0], hidden_dim*heads[1])
        self.conv3 = GATConv(hidden_dim*heads[1], out_dim, add_self_loops=True,heads = heads[2], dropout = dropout,concat=False)
        self.lin3 = Linear(hidden_dim*heads[1], out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = F.elu(x)
        x = self.conv2(x, edge_index) + self.lin2(x)
        x = F.elu(x)
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x

In [ ]:
import  matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from torcheval.metrics.functional import multiclass_f1_score
import torch 
import copy 
def train_and_evaluate_gnn(sets):     
    EPOCHS = 150 #200
    HIDDEN_DIM = 512 #64
    HEADS = [8,8,8]#[4,4,6]
    DROPOUT = 0.2
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 0
    
    
    model = GNN(in_dim=sets[TRAIN][0].x.shape[-1], hidden_dim=HIDDEN_DIM, out_dim=sets[TRAIN][0].y.shape[-1], heads = HEADS, dropout = DROPOUT).to(device)
    print(model)
    optim = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean')
    
    losses = dict()
    for set_name in set_names: losses[set_name] = []
    scores = dict()
    for set_name in set_names: scores[set_name] = []
    
    @torch.no_grad
    def validate(model, set_name, sets):
        model = copy.deepcopy(model)
        acc_loss = 0
        batch_size = 0
        ground_truth = []
        preds = []
        # with torch.inference_mode():
        model.eval()
        eval_set = sets[set_name]

        for loader in eval_set:
            loader = loader.to(device)
            out = model(loader.x, loader.edge_index)
            loss = loss_fn(out, loader.y)
            ground_truth.append(loader.y)
            preds.append((out > 0).float())
            acc_loss += loss.item()
            batch_size+=1
                
        score = f1_score(torch.cat(ground_truth).cpu(), torch.cat(preds).detach().cpu(), average ="micro")
        scores[set_name].append(score)
        
        losses[set_name].append(acc_loss / batch_size)
        return score
    
    def train(model):
        global PATIENCE_COUNT
        for epoch in tqdm(range(EPOCHS)):
            acc_loss = 0
            for loader in sets[TRAIN]:
                loader = loader.to(device)
                optim.zero_grad()
                model.train()                
                out = model(loader.x, loader.edge_index)
                loss = loss_fn(out, loader.y)
                acc_loss += loss.item()
                loss.backward()
                optim.step()
    start = time.time()
    train(model)
    elapsed_time = time.time() - start
    score = validate(model, VAL, sets)
    print(f"F1-score: {score}; Elapsed time: {elapsed_time} s.")
    return elapsed_time, score

In [94]:
unnormalized_train_dataset = PPI(root='/tmp/PPI', split="train")
unnormalized_val_dataset = PPI(root='/tmp/PPI', split="val")
unnormalized_test_dataset = PPI(root='/tmp/PPI', split="test")


unnormalized_data = []
unnormalized_data.extend(unnormalized_train_dataset)
unnormalized_data.extend(unnormalized_val_dataset)

In [95]:
TRAIN = "train"
VAL = "val"
set_names = [TRAIN, VAL]

device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

In [96]:
from sklearn.model_selection import KFold
import torch 

kf = KFold(n_splits=5)
gnn_scores = []
gnn_times = []
for i, (train_index, test_index) in enumerate(kf.split(unnormalized_data)):
    train_data = [unnormalized_data[i] for i in train_index]
    test_data = [unnormalized_data[i] for i in test_index]    
      
    sets = dict()
    sets[TRAIN] = train_data
    sets[VAL] = test_data
    
    elapsed_time, score = train_and_evaluate_gnn(sets)
    gnn_scores.append(score)
    gnn_times.append(elapsed_time)

GNN(
  (conv1): GATConv(50, 512, heads=8)
  (lin1): Linear(in_features=50, out_features=4096, bias=True)
  (conv2): GATConv(4096, 512, heads=8)
  (lin2): Linear(in_features=4096, out_features=4096, bias=True)
  (conv3): GATConv(4096, 121, heads=8)
  (lin3): Linear(in_features=4096, out_features=121, bias=True)
)


  0%|          | 0/150 [00:00<?, ?it/s]

F1-score: 0.983159619369414; Elapsed time: 258.689404964447 s.
GNN(
  (conv1): GATConv(50, 512, heads=8)
  (lin1): Linear(in_features=50, out_features=4096, bias=True)
  (conv2): GATConv(4096, 512, heads=8)
  (lin2): Linear(in_features=4096, out_features=4096, bias=True)
  (conv3): GATConv(4096, 121, heads=8)
  (lin3): Linear(in_features=4096, out_features=121, bias=True)
)


  0%|          | 0/150 [00:00<?, ?it/s]

F1-score: 0.9747386734407827; Elapsed time: 254.24147200584412 s.
GNN(
  (conv1): GATConv(50, 512, heads=8)
  (lin1): Linear(in_features=50, out_features=4096, bias=True)
  (conv2): GATConv(4096, 512, heads=8)
  (lin2): Linear(in_features=4096, out_features=4096, bias=True)
  (conv3): GATConv(4096, 121, heads=8)
  (lin3): Linear(in_features=4096, out_features=121, bias=True)
)


...

F1-score: 0.9848582894794099; Elapsed time: 252.3985493183136 s.
GNN(
  (conv1): GATConv(50, 512, heads=8)
  (lin1): Linear(in_features=50, out_features=4096, bias=True)
  (conv2): GATConv(4096, 512, heads=8)
  (lin2): Linear(in_features=4096, out_features=4096, bias=True)
  (conv3): GATConv(4096, 121, heads=8)
  (lin3): Linear(in_features=4096, out_features=121, bias=True)
)


  0%|          | 0/150 [00:00<?, ?it/s]

F1-score: 0.99540119927234; Elapsed time: 245.50580954551697 s.
GNN(
  (conv1): GATConv(50, 512, heads=8)
  (lin1): Linear(in_features=50, out_features=4096, bias=True)
  (conv2): GATConv(4096, 512, heads=8)
  (lin2): Linear(in_features=4096, out_features=4096, bias=True)
  (conv3): GATConv(4096, 121, heads=8)
  (lin3): Linear(in_features=4096, out_features=121, bias=True)
)


  0%|          | 0/150 [00:00<?, ?it/s]

F1-score: 0.9855634404689042; Elapsed time: 232.22667288780212 s.
